In [1]:
import pandas as pd
import datatable as dt

In [2]:
tr_df = dt.fread('../jane-street-market-prediction/train.csv').to_pandas()
features = [c for c in tr_df.columns if 'feature' in c]

In [3]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

removed_features = []

for feature in features:
    temp = tr_df[feature].rolling(200,center=True,min_periods=1).mean()
    temp.update(tr_df[feature])
    if temp.isna().sum() > 0:
        removed_features.append(feature)
    tr_df[feature] = temp

# print(f'removing {len(removed_features)} out of {len(features)} features.')
# for feature in removed_features:
#     features.remove(feature)

In [4]:
categories = [-1, 1]

outputs = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']

def to_categorical(val):
    val = min(categories, key=lambda x:abs(x-val))
    category = categories.index(val)
    return category

for output in outputs:
    tr_df[output+'_cat'] = tr_df[output].apply(lambda row: to_categorical(row))

In [5]:
import tensorflow as tf
import numpy as np

class TCNDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, output, train_timesteps=100, batch_size=128, n_channels=1, shuffle=False):
        self.data = data
        self.output = output
        self.train_timesteps = train_timesteps
        self.pred_timesteps = len(outputs)
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.signals = (len(self.data)-(self.train_timesteps+self.pred_timesteps))
        self.indexes = np.arange(self.signals-1)
        if shuffle:
            np.random.shuffle(self.indexes)
        
    def __len__(self):
        return int(np.floor( self.signals / self.batch_size )-1)
        
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        start_indicies = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(start_indicies)
        
        # Generate data
        X, y = self.__data_generation(start_indicies)

        return X, y
    
    def on_epoch_end(self):
        return
    
    def __data_generation(self, start_indicies):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.train_timesteps, self.n_channels))
        y = np.empty((self.batch_size, len(outputs)))
        
        # Generate sequence data
        for i, index in enumerate(start_indicies):
            
            #print(i, index, index+self.train_timesteps, index+self.train_timesteps+self.pred_timesteps, len(self.data))
            # Store sample
            X[i,:,:] = self.data[index:index+self.train_timesteps]

            # This is for sequence prediction
            y[i, :] = self.output[index]
            
            # This is for fluctuation prediction
            # d_ = (self.data[index+self.train_timesteps-1] - self.data[index+self.train_timesteps-2])/(self.data[index+self.train_timesteps-2])
            # d = (self.data[index+self.train_timesteps] - self.data[index+self.train_timesteps-1])/(self.data[index+self.train_timesteps-1])
            # y[i,:] = d - d_
                                                                                                             
        return X, y

In [6]:
import inspect
from tensorflow.keras import backend as K, Model, Input, optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, SpatialDropout1D, Lambda
from tensorflow.keras.layers import Layer, Conv1D, Dense, BatchNormalization, LayerNormalization

class ResidualBlock(Layer):

    def __init__(self,
                 dilation_rate: int,
                 nb_filters: int,
                 kernel_size: int,
                 padding: str,
                 activation: str = 'relu',
                 dropout_rate: float = 0,
                 kernel_initializer: str = 'he_normal',
                 use_batch_norm: bool = False,
                 use_layer_norm: bool = False,
                 use_weight_norm: bool = False,
                 **kwargs):
        """Defines the residual block for the WaveNet TCN
        Args:
            x: The previous layer in the model
            training: boolean indicating whether the layer should behave in training mode or in inference mode
            dilation_rate: The dilation power of 2 we are using for this residual block
            nb_filters: The number of convolutional filters to use in this block
            kernel_size: The size of the convolutional kernel
            padding: The padding used in the convolutional layers, 'same' or 'causal'.
            activation: The final activation used in o = Activation(x + F(x))
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
            use_batch_norm: Whether to use batch normalization in the residual layers or not.
            use_layer_norm: Whether to use layer normalization in the residual layers or not.
            use_weight_norm: Whether to use weight normalization in the residual layers or not.
            kwargs: Any initializers for Layer class.
        """

        self.dilation_rate = dilation_rate
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        self.kernel_initializer = kernel_initializer
        self.layers = []
        self.layers_outputs = []
        self.shape_match_conv = None
        self.res_output_shape = None
        self.final_activation = None

        super(ResidualBlock, self).__init__(**kwargs)

    def _build_layer(self, layer):
        """Helper function for building layer
        Args:
            layer: Appends layer to internal layer list and builds it based on the current output
                   shape of ResidualBlocK. Updates current output shape.
        """
        self.layers.append(layer)
        self.layers[-1].build(self.res_output_shape)
        self.res_output_shape = self.layers[-1].compute_output_shape(self.res_output_shape)

    def build(self, input_shape):

        with K.name_scope(self.name):  # name scope used to make sure weights get unique names
            self.layers = []
            self.res_output_shape = input_shape

            for k in range(2):
                name = 'conv1D_{}'.format(k)
                with K.name_scope(name):  # name scope used to make sure weights get unique names
                    conv = Conv1D(
                        filters=self.nb_filters,
                        kernel_size=self.kernel_size,
                        dilation_rate=self.dilation_rate,
                        padding=self.padding,
                        name=name,
                        kernel_initializer=self.kernel_initializer
                    )
                    if self.use_weight_norm:
                        from tensorflow_addons.layers import WeightNormalization
                        # wrap it. WeightNormalization API is different than BatchNormalization or LayerNormalization.
                        with K.name_scope('norm_{}'.format(k)):
                            conv = WeightNormalization(conv)
                    self._build_layer(conv)

                with K.name_scope('norm_{}'.format(k)):
                    if self.use_batch_norm:
                        self._build_layer(BatchNormalization())
                    elif self.use_layer_norm:
                        self._build_layer(LayerNormalization())
                    elif self.use_weight_norm:
                        pass  # done above.

                self._build_layer(Activation(self.activation))
                self._build_layer(SpatialDropout1D(rate=self.dropout_rate))

            if self.nb_filters != input_shape[-1]:
                # 1x1 conv to match the shapes (channel dimension).
                name = 'matching_conv1D'
                with K.name_scope(name):
                    # make and build this layer separately because it directly uses input_shape
                    self.shape_match_conv = Conv1D(filters=self.nb_filters,
                                                   kernel_size=1,
                                                   padding='same',
                                                   name=name,
                                                   kernel_initializer=self.kernel_initializer)
            else:
                name = 'matching_identity'
                self.shape_match_conv = Lambda(lambda x: x, name=name)

            with K.name_scope(name):
                self.shape_match_conv.build(input_shape)
                self.res_output_shape = self.shape_match_conv.compute_output_shape(input_shape)

            self._build_layer(Activation(self.activation))
            self.final_activation = Activation(self.activation)
            self.final_activation.build(self.res_output_shape)  # probably isn't necessary

            # this is done to force Keras to add the layers in the list to self._layers
            for layer in self.layers:
                self.__setattr__(layer.name, layer)
            self.__setattr__(self.shape_match_conv.name, self.shape_match_conv)
            self.__setattr__(self.final_activation.name, self.final_activation)

            super(ResidualBlock, self).build(input_shape)  # done to make sure self.built is set True

    def call(self, inputs, training=None):
        """
        Returns: A tuple where the first element is the residual model tensor, and the second
                 is the skip connection tensor.
        """
        x = inputs
        self.layers_outputs = [x]
        for layer in self.layers:
            training_flag = 'training' in dict(inspect.signature(layer.call).parameters)
            x = layer(x, training=training) if training_flag else layer(x)
            self.layers_outputs.append(x)
        x2 = self.shape_match_conv(inputs)
        self.layers_outputs.append(x2)
        res_x = layers.add([x2, x])
        self.layers_outputs.append(res_x)

        res_act_x = self.final_activation(res_x)
        self.layers_outputs.append(res_act_x)
        return [res_act_x, x]

    def compute_output_shape(self, input_shape):
        return [self.res_output_shape, self.res_output_shape]


In [7]:
class TCN(Layer):
    """Creates a TCN layer.
        Input shape:
            A tensor of shape (batch_size, timesteps, input_dim).
        Args:
            nb_filters: The number of filters to use in the convolutional layers. Can be a list.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual blocK.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            activation: The activation used in the residual blocks o = Activation(x + F(x)).
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            kernel_initializer: Initializer for the kernel weights matrix (Conv1D).
            use_batch_norm: Whether to use batch normalization in the residual layers or not.
            kwargs: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model.
                    Use unique names when using multiple TCN.
        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=(1, 2, 4, 8, 16, 32),
                 padding='causal',
                 use_skip_connections=False,
                 dropout_rate=0.0,
                 return_sequences=False,
                 activation='relu',
                 kernel_initializer='he_normal',
                 use_batch_norm=False,
                 use_layer_norm=False,
                 use_weight_norm=False,
                 **kwargs):

        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.activation = activation
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.use_batch_norm = use_batch_norm
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        self.skip_connections = []
        self.residual_blocks = []
        self.layers_outputs = []
        self.build_output_shape = None
        self.slicer_layer = None  # in case return_sequence=False
        self.output_slice_index = None  # in case return_sequence=False
        self.padding_same_and_time_dim_unknown = False  # edge case if padding='same' and time_dim = None

        if self.use_batch_norm + self.use_layer_norm + self.use_weight_norm > 1:
            raise ValueError('Only one normalization can be specified at once.')

        if isinstance(self.nb_filters, list):
            assert len(self.nb_filters) == len(self.dilations)

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

        # initialize parent class
        super(TCN, self).__init__(**kwargs)

    @property
    def receptive_field(self):
        assert_msg = 'The receptive field formula works only with power of two dilations.'
        assert all([is_power_of_two(i) for i in self.dilations]), assert_msg
        return self.kernel_size * self.nb_stacks * self.dilations[-1]

    def build(self, input_shape):

        # member to hold current output shape of the layer for building purposes
        self.build_output_shape = input_shape

        # list to hold all the member ResidualBlocks
        self.residual_blocks = []
        total_num_blocks = self.nb_stacks * len(self.dilations)
        if not self.use_skip_connections:
            total_num_blocks += 1  # cheap way to do a false case for below

        for s in range(self.nb_stacks):
            for i, d in enumerate(self.dilations):
                res_block_filters = self.nb_filters[i] if isinstance(self.nb_filters, list) else self.nb_filters
                self.residual_blocks.append(ResidualBlock(dilation_rate=d,
                                                          nb_filters=res_block_filters,
                                                          kernel_size=self.kernel_size,
                                                          padding=self.padding,
                                                          activation=self.activation,
                                                          dropout_rate=self.dropout_rate,
                                                          use_batch_norm=self.use_batch_norm,
                                                          use_layer_norm=self.use_layer_norm,
                                                          use_weight_norm=self.use_weight_norm,
                                                          kernel_initializer=self.kernel_initializer,
                                                          name='residual_block_{}'.format(len(self.residual_blocks))))
                # build newest residual block
                self.residual_blocks[-1].build(self.build_output_shape)
                self.build_output_shape = self.residual_blocks[-1].res_output_shape

        # this is done to force keras to add the layers in the list to self._layers
        for layer in self.residual_blocks:
            self.__setattr__(layer.name, layer)

        self.output_slice_index = None
        if self.padding == 'same':
            time = self.build_output_shape.as_list()[1]
            if time is not None:  # if time dimension is defined. e.g. shape = (bs, 500, input_dim).
                self.output_slice_index = int(self.build_output_shape.as_list()[1] / 2)
            else:
                # It will known at call time. c.f. self.call.
                self.padding_same_and_time_dim_unknown = True

        else:
            self.output_slice_index = -1  # causal case.
        self.slicer_layer = Lambda(lambda tt: tt[:, self.output_slice_index, :])

    def compute_output_shape(self, input_shape):
        """
        Overridden in case keras uses it somewhere... no idea. Just trying to avoid future errors.
        """
        if not self.built:
            self.build(input_shape)
        if not self.return_sequences:
            batch_size = self.build_output_shape[0]
            batch_size = batch_size.value if hasattr(batch_size, 'value') else batch_size
            nb_filters = self.build_output_shape[-1]
            return [batch_size, nb_filters]
        else:
            # Compatibility tensorflow 1.x
            return [v.value if hasattr(v, 'value') else v for v in self.build_output_shape]

    def call(self, inputs, training=None):
        x = inputs
        self.layers_outputs = [x]
        self.skip_connections = []
        for layer in self.residual_blocks:
            try:
                x, skip_out = layer(x, training=training)
            except TypeError:  # compatibility with tensorflow 1.x
                x, skip_out = layer(K.cast(x, 'float32'), training=training)
            self.skip_connections.append(skip_out)
            self.layers_outputs.append(x)

        if self.use_skip_connections:
            x = layers.add(self.skip_connections)
            self.layers_outputs.append(x)

        if not self.return_sequences:
            # case: time dimension is unknown. e.g. (bs, None, input_dim).
            if self.padding_same_and_time_dim_unknown:
                self.output_slice_index = K.shape(self.layers_outputs[-1])[1] // 2
            x = self.slicer_layer(x)
            self.layers_outputs.append(x)
        return x

    def get_config(self):
        """
        Returns the config of a the layer. This is used for saving and loading from a model
        :return: python dictionary with specs to rebuild layer
        """
        config = super(TCN, self).get_config()
        config['nb_filters'] = self.nb_filters
        config['kernel_size'] = self.kernel_size
        config['nb_stacks'] = self.nb_stacks
        config['dilations'] = self.dilations
        config['padding'] = self.padding
        config['use_skip_connections'] = self.use_skip_connections
        config['dropout_rate'] = self.dropout_rate
        config['return_sequences'] = self.return_sequences
        config['activation'] = self.activation
        config['use_batch_norm'] = self.use_batch_norm
        config['use_layer_norm'] = self.use_layer_norm
        config['use_weight_norm'] = self.use_weight_norm
        config['kernel_initializer'] = self.kernel_initializer
        return config

In [8]:
from tensorflow.keras import Input, optimizers, Model
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras import backend as K
#from tcn import TCN

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def build_tcn(max_len, num_feat, num_classes, lr=0.002):
    # Receptive field = nb_stacks_of_residuals_blocks * kernel_size * last_dilation.
    # --> 24 * 8 * 1 = 216
    input_layer = Input(shape=(max_len, num_feat))
    x = TCN(nb_filters=24, 
            kernel_size=8, 
            nb_stacks=1, 
            dilations=[2 ** i for i in range(8)],
            padding='causal',
            use_skip_connections=True, 
            dropout_rate=0.30, 
            return_sequences=False,
            activation='relu', 
            kernel_initializer='he_normal', 
            use_batch_norm=False, 
            use_layer_norm=False,
            name='tcn')(input_layer)
    opt = optimizers.Adam(lr=lr, clipnorm=1.)
    
    # classification
    x = Dense(num_classes)(x)
    x = Activation('sigmoid')(x)
    output_layer = x
    print(output_layer.shape)
    model = Model(input_layer, output_layer)
    model.compile(optimizers.Adam(lr=lr, clipnorm=1.), loss='categorical_crossentropy', metrics=['mae', 'accuracy'])
    return model

In [9]:
import matplotlib.pyplot as plt

def GetSpacedElements(array, numElems = 4):
    out = array[np.round(np.linspace(0, len(array)-1, numElems)).astype(int)]
    return out

In [10]:
def test_error_count(gen):
    x_error_count = 0
    y_error_count = 0
    for g_i in range(len(gen)+1):
        x, y = gen[g_i]
        for i in range(x.shape[0]):
            if np.mean(x[i,:,0]) < 5000:
                x_error_count += 1
            if y[i,0] < 5000:
                y_error_count += 1
    print(x_error_count, y_error_count)

In [11]:
epochs = 3
input_length = 250

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)
data = tr_df[features].values
output = tr_df[outputs].values
fold = 1

models = []

for trn_idx, tst_idx in tscv.split(data):
    
        print("Fold: {}".format(fold))
        gen = TCNDataGenerator(data[trn_idx],
                               output[trn_idx],
                               train_timesteps = input_length,
                               batch_size = 1024,
                               n_channels = len(features))
        model = build_tcn(num_feat=len(features), max_len=input_length, num_classes=len(outputs))
        
        print("Fitting...")
        # Verbosity set to 1 because it doesn't behave very well in Jupyter Notebook
        model.fit(gen, epochs=epochs, verbose=1)
        
        # Test Model
        gen = TCNDataGenerator(data[tst_idx],
                               output[tst_idx],
                               train_timesteps = input_length,
                               batch_size = 1024,
                               n_channels = len(features))
        
        print("Evalutating...")
        model.evaluate(gen, verbose=1)
        models.append(model)
        model.save_weights(f'{fold}_model.h5')
        fold = fold+1

Fold: 1
(None, 5)
Fitting...
Epoch 1/3
387/387 [==============================] - 159s 411ms/step - loss: 0.0049 - mae: 0.5014 - accuracy: 0.2620
Epoch 2/3
387/387 [==============================] - 155s 400ms/step - loss: 0.0048 - mae: 0.5044 - accuracy: 0.3097
Epoch 3/3
387/387 [==============================] - 156s 402ms/step - loss: 0.0048 - mae: 0.5210 - accuracy: 0.3037
Evalutating...
387/387 [==============================] - 92s 238ms/step - loss: 0.0032 - mae: 0.5279 - accuracy: 0.3613
Fold: 2
(None, 5)
Fitting...
Epoch 1/3
776/776 [==============================] - 333s 429ms/step - loss: 0.0041 - mae: 0.5188 - accuracy: 0.3495
Epoch 2/3
776/776 [==============================] - 298s 384ms/step - loss: 0.0040 - mae: 0.5344 - accuracy: 0.3450
Epoch 3/3
776/776 [==============================] - 296s 381ms/step - loss: 0.0040 - mae: 0.5460 - accuracy: 0.3549
Evalutating...
387/387 [==============================] - 87s 225ms/step - loss: 0.0019 - mae: 0.5632 - accuracy: 0.352